In [1]:
#---+----1----+----2----+----3----+----4----+----5----+----6----+----7----+----8

**Loop through the arcs in the month of november**
  1. Stage 1
     - for each day of interest, find the earliest epoch time in that day that matches the raw data
        - set each epoch_start value to be that time for each day, respectively
     - run GEODYN with the earliest time of day until the end of the day at midnight.
     - Do for 1 week
     - Collect the trajectory outputs into a file, saving just state vector of the starttime and endtime
    
  
  2. Stage 2  
     - Set the epoch start time to midnight
     - Use the final state vector values from the previous day (should be within 1 minute of midnight) as the new initial condition
  

In [2]:
from  datetime import datetime, timedelta
import pandas as pd
import numpy  as np

In [3]:
# days = np.arange(2,29+1)
days = np.arange(2,5+1)

## Stage 1

### find earliest daily start time to match data

In [4]:
start_0ut = []
end___0ut = []
arcs      = []
start_update = []
#
dt_1days = pd.Series(pd.to_timedelta(24,'h'))
#
for iday,dayval in enumerate(days):
    
    prms_arc={}
    prms_arc['epoch_start'] = f"2018-11-{dayval:02d} 00:00:00"
    prms_arc['epoch_startDT'] = pd.to_datetime(prms_arc['epoch_start'],\
                                                format='%Y-%m-%d %H:%M:%S')
    
    start_0ut.append(f"2018-11-{dayval:02d} 00:00:00")
    end___0ut.append(pd.to_datetime(prms_arc['epoch_startDT']+dt_1days).dt.strftime('%Y-%m-%d %H:%M:%S').values[0])
    arcs.append(prms_arc['epoch_startDT'].strftime('%Y.%j'))
    
    epoch_startDT = prms_arc['epoch_startDT']
    file_statevector_ICs = "/data/SatDragModelValidation/data/inputs/"\
                            +"sat_spire83/setups/Spire83_initialconditions_Nov2018_v1.txt"

    datetype = 'datetime_string'
    date_in_file_flag= False
    import linecache

    ### Only need to use accuracy to within 1 second (ignore the microseconds in the file)

    if datetype == 'datetime_string':
        date_str = str(epoch_startDT)
    elif datetype == 'YYMMDDHHMMSS':
        date_str = datetime.strftime(epoch_startDT, '%y%m%d%H%M%S')


    with open(file_statevector_ICs, 'r') as f:
        for line_no, line_text in enumerate(f):
            if date_str in line_text:
                date_in_file_flag= True
#                 print('    ','xyzline',line_no,line_text)
                break
    if date_in_file_flag == False:
#         print(date_str,'not found in file.')

        ### Find the dates that have the same hour    
        if datetype == 'datetime_string':
            date_roundhour_str = str(epoch_startDT)[:10]
        elif datetype == 'YYMMDDHHMMSS':
            date_roundhour_str = datetime.strftime(epoch_startDT, '%y%m%d%H')

#         print(date_roundhour_str)
        ### Scan through IC file and append a list of dates within the same hour
        line_no_list = []
        line_list = []
        with open(file_statevector_ICs, 'r') as f:
            for line_no, line_text in enumerate(f):
                if date_roundhour_str in line_text:
                    line_no_list.append(line_no)


        # print(line_no_list)
        for i in np.arange(line_no_list[1], line_no_list[-1]):
            line = linecache.getline(file_statevector_ICs,i)
            line_list.append(line)
        dates = []
        for i ,val in enumerate(line_list):
            if datetype == 'datetime_string':
                dates.append(pd.to_datetime(line_list[i][:19],format='%Y-%m-%d %H:%M:%S'))
            elif datetype == 'YYMMDDHHMMSS':
                dates.append(pd.to_datetime(line_list[i][:19],format='%y%m%d%H%M%S.%f'))
        
        start_update.append(dates[1])
        
    else:
#         print('Found date in IC file:', str(epoch_startDT))
        xyzline = pd.read_csv(file_statevector_ICs, 
                    skiprows = line_no, 
                    nrows=1,           
                    sep = '\s+',
                    dtype=str,
                    names = [
                        'DateYMD',
                        'DateHMS',
                        'X',
                        'Y',
                        'Z',
                        'X_dot',
                        'Y_dot',
                        'Z_dot',
                            ],)
        start_update.append(pd.to_datetime(xyzline['DateYMD']+xyzline['DateHMS'], format='%Y-%m-%d%H:%M:%S')[0])

    print(prms_arc['epoch_start'][:10], "earliest time is:", start_update[iday])

#         X     =  float(xyzline['X'].values[0].ljust(20))     #'  -745933.8926940708'
#         Y     =  float(xyzline['Y'].values[0].ljust(20))     #'  -4864983.834066438'
#         Z     =  float(xyzline['Z'].values[0].ljust(20))     #'    4769954.60524261'
#         X_dot =  float(xyzline['X_dot'].values[0].ljust(20)) #'  457.44564954037634'
#         Y_dot =  float(xyzline['Y_dot'].values[0].ljust(20)) #'   5302.381564886811'
#         Z_dot =  float(xyzline['Z_dot'].values[0].ljust(20)) #'    5463.55571622269'
#     print(f"   [X,Y,Z]:          [{X    :15.5f}, {Y    :15.5f}, {Z    :15.5f}]")
#     print(f"   [Xdot,Ydot,Zdot]: [{X_dot:15.5f}, {Y_dot:15.5f}, {Z_dot:15.5f}]")        
#     print()        

del date_roundhour_str
del line_no_list
del line_list
del dates
del xyzline

start_update = [datetime.strftime(idate, '%Y-%m-%d %H:%M:%S') for idate in start_update]


2018-11-02 earliest time is: 2018-11-02 00:00:00
2018-11-03 earliest time is: 2018-11-03 01:02:55
2018-11-04 earliest time is: 2018-11-04 00:52:55
2018-11-05 earliest time is: 2018-11-05 00:45:16


In [5]:
print(start_0ut)
print(end___0ut)
print(start_update)
print(arcs)


['2018-11-02 00:00:00', '2018-11-03 00:00:00', '2018-11-04 00:00:00', '2018-11-05 00:00:00']
['2018-11-03 00:00:00', '2018-11-04 00:00:00', '2018-11-05 00:00:00', '2018-11-06 00:00:00']
['2018-11-02 00:00:00', '2018-11-03 01:02:55', '2018-11-04 00:52:55', '2018-11-05 00:45:16']
['2018.306', '2018.307', '2018.308', '2018.309']


### Run GEODYN with earliest start time to midnight

In [6]:
%load_ext autoreload
%autoreload 2

from pygeodyn.PYGEODYN import Pygeodyn
from gc import collect as gc_collect

# run_settings = Pygeodyn.print_input(options=False, info=False)


settings_SPIRE= {# Basic input settings
                 'satellite'      : {'input': 'spire83'},
                 'den_model'      : {'input': 'jb2008'},
                 'run_type'       : {'input': 'DataReduction_PCE'},
                 'run_specifier'  : {'input': '_test_infrastruc'},
                 'cd_model'       : {'input': 'BWDRAG'},
                 'file_string'    : {'input': 'CD_2p3'},
                 # Force Model settings
                  'cd_type'               : {'input':"Fixed_CD"},
                  'cd_value'              : {'input':2.300000},
                  'scaling_factor'        : {'input':False},
                  'cd_adjustment_boolean' : {'input':False },
                  'hours_between_cd_adj'  : {'input':6 },
                 # Run
                  'step'           : {'input': 30.},
                  'orbfil_step'    : {'input': 30.},    
                   #
                  'arc'            : {'input':arcs},
                  'epoch_start'    : {'input':start_update},
                   #
                  'epoch_stop'     : {'input':end___0ut},       
                   #
                  'global_options' : {'input':'pso_2018'},
                 # Request read on raw outputs
                  'request_data'   : {'input': ['Trajectory_orbfil']},
              #end dict
              }

In [7]:
sat = Pygeodyn(settings_SPIRE, use_file=False)

# sat.run_arcs()
obj = sat.getData()


2018.306,  loading jb2008
  - convergence achieved
     xyzline 30754 2018-11-02 00:00:00   -4520742.22058   -1301871.58945    4986886.37752       -394.47597      -7276.54789      -2244.24756

Found date in IC file: 2018-11-02 00:00:00
   [X,Y,Z]:          [ -4520742.22058,  -1301871.58945,   4986886.37752]
   [Xdot,Ydot,Zdot]: [     -394.47597,     -7276.54789,     -2244.24756]


2018.307,  loading jb2008
  - convergence achieved
     xyzline 64658 2018-11-03 01:02:55   -4242037.15803     740496.48895    5331691.84664      -2271.36157      -7237.31513       -792.19468

Found date in IC file: 2018-11-03 01:02:55
   [X,Y,Z]:          [ -4242037.15803,    740496.48895,   5331691.84664]
   [Xdot,Ydot,Zdot]: [    -2271.36157,     -7237.31513,      -792.19468]


2018.308,  loading jb2008
  - convergence achieved
     xyzline 97343 2018-11-04 00:52:55   -4004970.31658   -5385662.24386    1437820.04371       3129.31949      -3877.98799      -5766.28836

Found date in IC file: 2018-11-04 00:5

### Load the data, grab the final trajectory point from the previous day, use this value as initial condition.

In [8]:
IC_update_0ut=[]

for iarc, valarc in enumerate(obj.__dict__['global_params']['arc_input']):
    print('Writing values for', valarc)
    IC_update_0ut.append([\
            obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['X j2000'].values[-1],
            obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Y j2000'].values[-1],
            obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Z j2000'].values[-1],
            obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['X_dot j2000'].values[-1],
            obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Y_dot j2000'].values[-1],
            obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Z_dot j2000'].values[-1]
                        ])


Writing values for 2018.306
Writing values for 2018.307
Writing values for 2018.308
Writing values for 2018.309


In [9]:
print(IC_update_0ut)

[[523110.1443382325, -5930104.572804029, -3421833.1925745346, 5184.988481638479, 3125.362395719306, -4620.351856357143], [4725171.38309032, 3834594.807539863, -3161748.204033614, -1402.6360786755376, 5715.506672554241, 4849.866163957699], [-2121797.632386333, 4137298.3437206373, 5030508.754469125, -5154.27310355572, -5213.25900586897, 2117.2400239433664], [-4397963.726929919, -5242051.208765718, 558498.358406628, 3308.0804952364015, -3421.299515092137, -5950.286314239187]]


In [10]:
# # update_0ut_sv=[]

# # for iarc, valarc in enumerate(obj.__dict__['global_params']['arc_input']):
# #     print()
# # #     print(obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Date_UTC'].values[-1])
# # #     print('   ',obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['X j2000'].values[-1])
# #     update_0ut_sv.append([\
# #             obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['X j2000'].values[-1],
# #             obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Y j2000'].values[-1],
# #             obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Z j2000'].values[-1],
# #             obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['X_dot j2000'].values[-1],
# #             obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Y_dot j2000'].values[-1],
# #             obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Z_dot j2000'].values[-1]
# #                         ])
    
# update_0ut_sv = [  [ 3928916.458133645 ,  #[3814035.3855300485,
#                    -1872640.264047095 ,  # -2032967.592959765,
#                    -5308435.441564657 ,  # -5332424.200973777,
#                     4321.568196942277 ,  # 4445.699155052163,
#                     6185.181504378663 ,  # 6122.880673353949,
#                     1024.611110847065],  # 853.7335509694938],
#                 [3338067.1976274904,
#                 5588610.712440854,
#                 2136698.256099367,
#                 -5201.3580624253755,
#                 996.740676696283,
#                 5498.429266661564],
#                 [-5426338.454433531,
#                 -169513.06801732714,
#                 4187973.912968636,
#                 -2720.9192328067647,
#                 -6055.799912547885,
#                 -3750.909866748835],
#                 [-1361128.728764797,
#                 -5124562.117802262,
#                 -4364950.955087397,
#                 6647.617864337456,
#                 1221.4949833863877,
#                 -3502.5319366388662],
#                 [6314428.287261461,
#                 1998038.7321490748,
#                 -1770479.5953752713,
#                 -44.52103320599878,
#                 5118.362538082618,
#                 5657.972765270434],
#                 [-1393576.3065223494,
#                 4026748.015999339,
#                 5361457.6461432865,
#                 -7032.783574258777,
#                 -2938.4411141419014,
#                 385.7254176550402],
#                 [-6010694.155270688,
#                 -3132233.352653585,
#                 -1096273.1800164438,
#                 3044.0518607378153,
#                 -3807.468037176313,
#                 -5855.0109650008535],
#                 ]

# # [update_0ut_sv[0],
# # update_0ut_sv[1],
# # update_0ut_sv[2],
# # update_0ut_sv[3],
# # update_0ut_sv[4],
# # update_0ut_sv[5],
# # ]

In [11]:
arcs

['2018.306', '2018.307', '2018.308', '2018.309']

In [12]:
arcs[:-1]

['2018.306', '2018.307', '2018.308']

In [13]:
%load_ext autoreload
%autoreload 2

from pygeodyn.PYGEODYN import Pygeodyn
from gc import collect as gc_collect

# run_settings = Pygeodyn.print_input(options=False, info=False)


settings_SPIRE= {# Basic input settings
                 'satellite'      : {'input': 'spire83'},
                 'den_model'      : {'input': 'jb2008'},
                 'run_type'       : {'input': 'DataReduction_PCE'},
                 'run_specifier'  : {'input': '_test_ics'},
                 'cd_model'       : {'input': 'BWDRAG'},
                 'file_string'    : {'input': 'CD_2p3'},
               # Force Model settings
                  'cd_type'               : {'input':"Fixed_CD"},
                  'cd_value'              : {'input':2.300000},
                  'scaling_factor'        : {'input':False},
                  'cd_adjustment_boolean' : {'input':False },
                  'hours_between_cd_adj'  : {'input':6 },
               # Run
                  'step'           : {'input': 60.},
                  'orbfil_step'    : {'input': 60.},    

                  #
                  'arc'            : {'input':arcs[1:]},
                  'epoch_start'    : {'input':start_0ut[1:]},
                  'epoch_stop'     : {'input':end___0ut[1:]},       
                  'initial_conditions':{'input':IC_update_0ut[:-1]},

                  'global_options' : {'input':'pso_2018'},
               # Request read on raw outputs
                  'request_data'   : {'input': ['Trajectory_orbfil', 
                                                'RunSummary']},
              #end dict
              }

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
sat = Pygeodyn(settings_SPIRE, use_file=False)

sat.run_arcs()
obj = sat.getData()

G2B file exists: /data/SatDragModelValidation/data/inputs/sat_spire83/g2b/g2b_pce_leoOrb_nov2018
Run #1     Current Time =      09:51:31  GMT-7
Run #1
|————————————————————————————————————————————————————————————————————————————————
| Run #1 Parameters
| —————————————————
|  Run Specs
|  --------- 
|    Satellite    spire83
|    Run Type     DataReduction_PCE
|    CD Type      Fixed_CD
|    CD Value     2.3
|    Density      jb2008
|
|  Epoch Info 
|  ---------- 
|    Arc          2018.307
|    Arc length   24.0 hours
|    Epoch Start  2018-11-03 00:00:00
|    Epoch End    2018-11-04 00:00:00
|    Step Size    60.0 seconds
|
|  Files Info
|  ---------- 
|    Arc Name    spire83_2018307.01_jb2008.CD_2p3
|
|    IISSET      .../data/inputs/sat_spire83/setups/iisset.2018.307
|    EXAT        .../data/inputs/sat_spire83/external_attitude/EXAT01.2018.307.gz
|    Output Raw  .../data/outputs_raw/spire83/jb2008/jb2008_BWDRAG_test_ics/
|
|————————————————————————————————————————————————————————

In [15]:
# indx = 0
# arc = settings_SPIRE['arc']['input'][indx]

# # obj.__dict__['RunSummary'][arc]

# # 
# print(obj.__dict__['Trajectory_orbfil'][arc]['data_record']['X j2000'].values[0],
#       obj.__dict__['Trajectory_orbfil'][arc]['data_record']['Y j2000'].values[0],
#       obj.__dict__['Trajectory_orbfil'][arc]['data_record']['Z j2000'].values[0],
#       obj.__dict__['Trajectory_orbfil'][arc]['data_record']['X_dot j2000'].values[0],
#       obj.__dict__['Trajectory_orbfil'][arc]['data_record']['Y_dot j2000'].values[0],
#       obj.__dict__['Trajectory_orbfil'][arc]['data_record']['Z_dot j2000'].values[0])


# Finally grab values along these orbits and make these the Initial Conditions

In [16]:
# obj.__dict__['Trajectory_orbfil'][arc]['data_record'].columns


In [17]:
### Write to an ascii text file. 
file_save = '/data/SatDragModelValidation/data/inputs/sat_spire83/setups'\
                + '/Spire83_initialconditions_Nov2018_v2.txt'
arc0 = settings_SPIRE['arc']['input'][0]
with open(file_save, 'r+') as file:
    #### Manually write the header units
    header_units =\
                f"{'UTC'.rjust(len(str(obj.__dict__['Trajectory_orbfil'][arc0]['data_record']['Date_UTC'][1]))-1,' ') }"\
            +f"  {'(m)'.rjust(15,' ')}"\
            +f"  {'(m)'.rjust(15,' ')}"\
            +f"  {'(m)'.rjust(15,' ')}"\
            +f"  {'(m/s)'.rjust(15,' ')}"\
            +f"  {'(m/s)'.rjust(15,' ')}"\
            +f"  {'(m/s)'.rjust(15,' ')}"\

    #### Manually write the header field names
    header_names =\
                f"{'Date'.rjust(len(str(obj.__dict__['Trajectory_orbfil'][arc0]['data_record']['Date_UTC'][1]))-1,' ') }"\
            +f"  {'X'.rjust(15,' ')}"\
            +f"  {'Y'.rjust(15,' ')}"\
            +f"  {'Z'.rjust(15,' ')}"\
            +f"  {'X_dot'.rjust(15,' ')}"\
            +f"  {'Y_dot'.rjust(15,' ')}"\
            +f"  {'Z_dot'.rjust(15,' ')}"\


#---+----1----+----2----+----3----+----4----+----5----+----6----+----7----+----8
    #### Manually write the detailed header description
    header_meta = \
    f'''### Initial conditions file
### -----------------------
###     Satellite: Spire_083 (1804607)
###     Last modified: {datetime.now()-timedelta(hours=7)}
###
### Source
### -------
###     Initial conditions from Converged Solutions of GEODYN data reduction.
###     Constructed as follows:
###        Stage 1
###           - for each day of interest, find the earliest epoch time in that
###             day that matches the raw data (PCE)
###           - set epoch_start value to be that time for each day, respectively
###           - run GEODYN with the earliest time of day until midnight.
###           - Collect the trajectory outputs into a file, saving just
###             state vector of the starttime and endtime
###        Stage 2  
###           - Set the epoch start time to midnight
###           - Use the final state vector values from the previous day
###             (should be within 1 minute of midnight) as the new
###             initial condition
###           - Run geodyn for each daily arc from 0 UT to 0 UT
###        Stage 3 
###           - correct any failed convergences
###           - use the output orbit solutions from the converged solutions
###             as the updated initial conditions
###      note-- Orbits were fit to PCE (SpireLeoOrb 'POD' from RTOrb) 
###             using Jb2008 density model.
###
### Contents
### --------
###     Date: (YYYY-MM-DD hh:mm:ss.ssssss) (UTC)
###     pvi: Position and velocity (X, Y, Z, X_dot, Y_dot, Z_dot)
###          coordinate: ECI-J2000
###          unit: m and m/s
###
#{header_units}
#{header_names}
### %eoh
'''
    file.write(header_meta)


    for iarc, valarc in enumerate(obj.__dict__['global_params']['arc_input']):
        print('Writing values for', valarc)
    #     update_0ut_sv.append([\
    #             obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['X j2000'].values[-1],
    #             obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Y j2000'].values[-1],
    #             obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Z j2000'].values[-1],
    #             obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['X_dot j2000'].values[-1],
    #             obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Y_dot j2000'].values[-1],
    #             obj.__dict__['Trajectory_orbfil'][valarc]['data_record']['Z_dot j2000'].values[-1]
    #                         ])

        # 
        #
        orbit_data = obj.__dict__['Trajectory_orbfil'][valarc]['data_record']



        for ii, val in enumerate(orbit_data['Date_UTC']):
            ### Only print the hourly values (when the minute value is 0)
#             if orbit_data['Date_UTC'][ii].minute==0:
            row =   f"{orbit_data['Date_UTC'][ii]}"         \
                   +f"  {orbit_data['X j2000'][ii]:15.5f}"    \
                   +f"  {orbit_data['Y j2000'][ii]:15.5f}"    \
                   +f"  {orbit_data['Z j2000'][ii]:15.5f}"    \
                   +f"  {orbit_data['X_dot j2000'][ii]:15.5f}"\
                   +f"  {orbit_data['Y_dot j2000'][ii]:15.5f}"\
                   +f"  {orbit_data['Z_dot j2000'][ii]:15.5f}"\
                   + f"\n"
        #     print(row)
            file.write(row)




Writing values for 2018.307
Writing values for 2018.308
Writing values for 2018.309


In [18]:

# X_prop = obj.__dict__['Trajectory_orbfil'][arc]['data_record']['X j2000']
# Y_prop = obj.__dict__['Trajectory_orbfil'][arc]['data_record']['Y j2000']
# Z_prop = obj.__dict__['Trajectory_orbfil'][arc]['data_record']['Z j2000']
# #
# Xdot_prop = obj.__dict__['Trajectory_orbfil'][arc]['data_record']['X_dot j2000']
# Ydot_prop = obj.__dict__['Trajectory_orbfil'][arc]['data_record']['Y_dot j2000']
# Zdot_prop = obj.__dict__['Trajectory_orbfil'][arc]['data_record']['Z_dot j2000']

# date_prop = obj.__dict__['Trajectory_orbfil'][arc]['data_record']['Date_UTC']

# print(X_prop.values[0],
# Y_prop.values[0],
# Z_prop.values[0],
# Xdot_prop.values[0],
# Ydot_prop.values[0],
# Zdot_prop.values[0],)

In [19]:
#  'X POS': 3928931.51724803,
#  'Y POS': -1872620.338460264,
#  'Z POS': -5308433.202954784,
#  'X VEL': 4321.552827772032,
#  'Y VEL': 6185.186740784374,
#  'Z VEL': 1024.630175475583,


In [20]:
# epoch_start = settings_SPIRE['epoch_start']['input'][indx]


# prms_arc={}
# prms_arc['epoch_start'] = epoch_start
# prms_arc['epoch_startDT'] = pd.to_datetime(prms_arc['epoch_start'],\
#                                             format='%Y-%m-%d %H:%M:%S')

# epoch_startDT = prms_arc['epoch_startDT']
# file_statevector_ICs = "/data/SatDragModelValidation/data/inputs/"\
#                         +"sat_spire83/setups/Spire83_initialconditions_Nov2018_v1.txt"

# datetype = 'datetime_string'

# date_in_file_flag= False
# import linecache

# ### Only need to use accuracy to within 1 second (ignore the microseconds in the file)

# if datetype == 'datetime_string':
#     date_str = str(epoch_startDT)
# elif datetype == 'YYMMDDHHMMSS':
#     date_str = datetime.strftime(epoch_startDT, '%y%m%d%H%M%S')


# with open(file_statevector_ICs, 'r') as f:
#     ### Find the dates that have the same hour    
#     if datetype == 'datetime_string':
#         date_roundhour_str = str(epoch_startDT)[:10]
#     elif datetype == 'YYMMDDHHMMSS':
#         date_roundhour_str = datetime.strftime(epoch_startDT, '%y%m%d%H%M%S')

#     ### Scan through IC file and append a list of dates within the same hour
#     line_no_list = []
#     line_list = []
#     with open(file_statevector_ICs, 'r') as f:
#         for line_no, line_text in enumerate(f):
#             if date_roundhour_str in line_text:
#                 line_no_list.append(line_no)
#     for i in np.arange(line_no_list[0]-10, line_no_list[-1]+10):
#         line = linecache.getline(file_statevector_ICs,i)
#         line_list.append(line)
#     dates = []
#     for i ,val in enumerate(line_list):
#         if datetype == 'datetime_string':
#             dates.append(pd.to_datetime(line_list[i][:19],format='%Y-%m-%d %H:%M:%S'))
#         elif datetype == 'YYMMDDHHMMSS':
#             dates.append(pd.to_datetime(line_list[i][:19],format='%y%m%d%H%M%S.%f'))
            
            
            
# xyzline = pd.read_csv(file_statevector_ICs, 
#             skiprows = line_no_list[0], 
#             nrows=line_no_list[-1]- line_no_list[0],           
#             sep = '\s+',
#             dtype=object,
#             names = [
#                 'DateYMD',
#                 'DateHMS',
#                 'X',
#                 'Y',
#                 'Z',
#                 'X_dot',
#                 'Y_dot',
#                 'Z_dot',
#                     ],)

# xyzline['Date'] =  pd.to_datetime(xyzline['DateYMD']+xyzline['DateHMS'], format='%Y-%m-%d%H:%M:%S')
# del xyzline['DateYMD'], xyzline['DateHMS']







In [21]:
# line_no_list[0]

In [22]:
# import plotly.graph_objects as go
# from plotly.offline import plot, iplot
# from plotly.subplots import make_subplots
# import plotly.express as px

# config = dict({
#                 'displayModeBar': True,
#                 'responsive': False,
#                 'staticPlot': True,
#                 'displaylogo': False,
#                 'showTips': False,
#                 })


# fig = make_subplots(rows=3, cols=1)



# fig.add_trace(go.Scattergl(
#             x=pd.to_datetime(xyzline['Date']),
#             y=xyzline['X'].values.astype(float),
#                 mode='markers',
#                 opacity=1,
#                 marker=dict(size=3, color="blue"),
#                 showlegend=False),
#                 secondary_y=False, row=1, col=1,)

# fig.add_trace(go.Scattergl(
#             x=pd.to_datetime(xyzline['Date']),
#             y=xyzline['Y'].values.astype(float),
#                 mode='markers',
#                 opacity=1,
#                 marker=dict(size=3, color="blue"),
#                 showlegend=False),
#                 secondary_y=False, row=2, col=1,)

# fig.add_trace(go.Scattergl(
#             x=pd.to_datetime(xyzline['Date']),
#             y=xyzline['Z'].values.astype(float),
#                 mode='markers',
#                 opacity=1,
#                 marker=dict(size=3, color="blue"),     
#                 showlegend=False),
#                 secondary_y=False, row=3, col=1,)

# ### ==============================================================


# fig.add_trace(go.Scattergl(
#             x=pd.to_datetime(date_prop),
#             y=X_prop.values.astype(float),
#                 mode='markers',
#                 opacity=1,
#                 marker=dict(size=3, color='red'),
#                 showlegend=False),
#                 secondary_y=False, row=1, col=1,)

# fig.add_trace(go.Scattergl(
#             x=pd.to_datetime(date_prop),
#             y=Y_prop.values.astype(float),
#                 mode='markers',
#                 opacity=1,
#                 marker=dict(size=3, color='red'),
#                 showlegend=False),
#                 secondary_y=False, row=2, col=1,)

# fig.add_trace(go.Scattergl(
#             x=pd.to_datetime(date_prop),
#             y=Z_prop.values.astype(float),
#                 mode='markers',
#                 opacity=1,
#                 marker=dict(size=3, color='red'),     
#                 showlegend=False),
#                 secondary_y=False, row=3, col=1,)

# fig.update_yaxes( title="X",  exponentformat= 'power',row=1, col=1)
# fig.update_yaxes( title="Y",  exponentformat= 'power',row=2, col=1)
# fig.update_yaxes( title="Z",  exponentformat= 'power',row=3, col=1)


# fig.update_layout(title='PCE (blue, Spire LeoOrb POD); Orbit Fit (red, data reduction with PCE)',
#         autosize=True
# #         width=800,height=900,
#                 )
# fig.show(config=config)
